In [1]:
import requests as rq
from urllib.parse import urljoin
import pandas as pd
import tqdm

In [2]:
URL = "https://www.filmweb.pl"

In [3]:
class FilmwebClient:
    def __init__(self, server) -> None:
        self.server = server
        self.common_headers = {'x-locale': 'en'}
        self.users = []
        self.critics = []
        self.movies = []


    def search_movies(self, page: int):
        url = urljoin(self.server, f'/api/v1/films/search?page={page}')
        return rq.get(url, headers=self.common_headers)


    def get_critics(self):
        url = urljoin(self.server, '/api/v1/film/critics')
        return rq.get(url, headers=self.common_headers)
    

    def get_movie_rating(self, movie_id):
        url = urljoin(self.server, f'/api/v1/film/{movie_id}/rating')
        return rq.get(url, headers=self.common_headers)
    

    def get_movie_critics(self, movie_id):
        url = urljoin(self.server, f'/api/v1/film/{movie_id}/critics')
        return rq.get(url, headers=self.common_headers)
    

    def get_movie_review(self, movie_id, critic_username):
        url = urljoin(self.server, f'/api/v1/user/{critic_username}/vote/film/{movie_id}')
        return rq.get(url, headers=self.common_headers)

    def get_reviewed_movies(self, critic_username):
        url = urljoin(self.server, f'/api/v1/user/{critic_username}/vote/film/')
        return rq.get(url, headers=self.common_headers)
    
    def get_reviewed_servies(self, critic_username):
        url = urljoin(self.server, f'/api/v1/user/{critic_username}/vote/serial')
        return rq.get(url, headers=self.common_headers)
    
    def get_tv_series_review(self, series_id, critic_username):
        url = urljoin(self.server, f'/api/v1/user/{critic_username}/vote/serial/{series_id}')
        return rq.get(url, headers=self.common_headers)

In [4]:
filmweb = FilmwebClient(URL)

# Get movies

In [ ]:
def generator():
  while True:
    yield


page = 1
movies = []
for _ in tqdm.tqdm(generator()):
    response = filmweb.search_movies(page)
    if response.status_code != 200:
        print(f'Reached the end of movies list, last page: {page-1}, number of movies: {len(movies)}')
        break
    try:
        movies += response.json()['searchHits']
    except:
        print(f'Cannot parse movies from page {page}.')
    page+=1

Don't worry about above problem as long as the number of pages is reasonable.

In [8]:
df_movies = pd.DataFrame(movies)
df_movies.head()

,id,type
0,862,film
1,1048,film
2,998,film
3,671,film
4,9136,film


In [10]:
df_movies.count()

id      4630
type    4630
dtype: int64

Save so far collected data, just in case

In [69]:
df_movies[['id', 'type']].to_csv('intermidate/movies.csv', index=False)

# Get critics

In [14]:
df_critics = pd.DataFrame(filmweb.get_critics().json())
df_critics.head()

,id,userName,priority,publisher
0,1164541,michaloleszczyk,1,Spoiler Master Podcast
1,1243927,tru3,1,Magazyn SFP
2,1418184,Cateborough,1,POPcast - podcast o popkulturze
3,1589044,darek_arest,1,krytyk niezależny
4,3281021,smolinskisebastian,1,krytyk niezależny


In [15]:
df_critics.groupby('priority').count()

,id,userName,publisher
priority,,,
1,9,9,9
2,10,10,10
3,13,13,13
4,17,17,17
5,7,7,7
6,10,10,10
7,1,1,1


In [16]:
df_critics.count()

id           67
userName     67
priority     67
publisher    67
dtype: int64

In [77]:
df_critics.to_csv('intermidate/critics.csv', index=False)

# Dataset preparation
Iterate over critics and get movies which they reviewed

In [144]:
results = []
critics = df_critics['userName'].values.tolist()

In [145]:
it = 0
while len(results) < len(df_critics['userName']):
    for critic in tqdm.tqdm(critics[it:]):
        try:
            movies_ids = list(map(lambda response: {'username': critic, 'movie_id': response[0]}, filmweb.get_reviewed_movies(critic).json()))
            results += movies_ids
        except Exception as e:
            print(f'Exception when getting {critic}\'s reviews. {e}')
            break
        it+=1

100%|██████████| 67/67 [00:10<00:00,  6.37it/s]


In [147]:
df_critics_reviews = pd.DataFrame(results)
df_critics_reviews.head()

,username,movie_id
0,michaloleszczyk,10014103
1,michaloleszczyk,10025735
2,michaloleszczyk,805204
3,michaloleszczyk,877700
4,michaloleszczyk,10026692


In [169]:
for i in tqdm.tqdm(df_critics_reviews.values[2000:]):
    pass

100%|██████████| 4324/4324 [00:00<00:00, 3106040.50it/s]


In [170]:
it = 0
results = []
while len(results) < len(df_critics_reviews.values) or it >= len(df_critics_reviews.values):
    for case in tqdm.tqdm(df_critics_reviews.values[it:]):
        try:
            response = filmweb.get_movie_review(critic_username=case[0], movie_id=case[1]).json()
            if 'comment' in response:
                movie_rating = filmweb.get_movie_rating(case[1]).json()
                results.append({'username': case[0], 
                                'movie_id': case[1],
                                'user_rate': response['rate'],
                                'user_comment': response['comment'],
                                'movie_rating': movie_rating['rate'],
                                'movie_rating_count': movie_rating['count']
                                })
        except Exception as e:
            print(f'Fetching data about movie rating failed: {e}')
            break
        it+=1

  3%|▎         | 165/6324 [03:00<1:52:10,  1.09s/it]


Fetching data about movie rating failed: HTTPSConnectionPool(host='www.filmweb.pl', port=443): Max retries exceeded with url: /api/v1/user/tru3/vote/film/10019460 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7fc2157464a0>, 'Connection to www.filmweb.pl timed out. (connect timeout=None)'))


  5%|▍         | 295/6159 [04:09<1:22:46,  1.18it/s] 


Fetching data about movie rating failed: HTTPSConnectionPool(host='www.filmweb.pl', port=443): Max retries exceeded with url: /api/v1/film/164870/rating (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7fc215745db0>, 'Connection to www.filmweb.pl timed out. (connect timeout=None)'))


  3%|▎         | 176/5864 [02:45<1:29:21,  1.06it/s]


...

Fetching data about movie rating failed: HTTPSConnectionPool(host='www.filmweb.pl', port=443): Max retries exceeded with url: /api/v1/user/MCh092788/vote/film/695409 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7fc215744f70>, 'Connection to www.filmweb.pl timed out. (connect timeout=None)'))


 18%|█▊        | 36/195 [02:17<10:06,  3.81s/it]


Fetching data about movie rating failed: HTTPSConnectionPool(host='www.filmweb.pl', port=443): Max retries exceeded with url: /api/v1/user/MCh092788/vote/film/31672 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7fc2157456c0>, 'Connection to www.filmweb.pl timed out. (connect timeout=None)'))


100%|██████████| 159/159 [00:40<00:00,  3.90it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]


KeyboardInterrupt: 

In [171]:
ds_df = pd.DataFrame(results)
ds_df.head()

,username,movie_id,user_rate,user_comment,movie_rating,movie_rating_count
0,michaloleszczyk,10014103,8,Willa Hössa jako makieta najciemniejszej stron...,8.32095,296
1,michaloleszczyk,10025735,3,"Zgentryfikowana ""Mama i dziwka"" o smaku pumpki...",6.41777,833
2,michaloleszczyk,805204,9,"""Ile wilków widzisz na obrazku?"" A ile widzisz...",7.41684,9884
3,michaloleszczyk,10034333,9,Film bliski perfekcji; czysty i syty jak ronde...,7.36086,557
4,michaloleszczyk,10015490,9,Wielka niespodzianka i najbardziej oryginalny ...,5.44533,375


In [172]:
ds_df.count()

username              3057
movie_id              3057
user_rate             3057
user_comment          3057
movie_rating          3057
movie_rating_count    3057
dtype: int64

In [179]:
ds_df.to_json('../dataset/dataset_movies.json')

In [180]:
ds_df.to_csv('../dataset/dataset_movies.csv', index=False, sep='$', encoding='utf8')

TODO: fetch tv series analogously to movies
DO NOT MERGE THE DATASETS! We may consider if we want to train model only on movies, tv series or both
# TV series

In [5]:
df_critics = pd.read_csv('intermidate/critics.csv')
df_critics.head()

,Unnamed: 0,id,userName,priority,publisher
0,0,1164541,michaloleszczyk,1,Spoiler Master Podcast
1,1,1243927,tru3,1,Magazyn SFP
2,2,1418184,Cateborough,1,POPcast - podcast o popkulturze
3,3,1589044,darek_arest,1,krytyk niezależny
4,4,3281021,smolinskisebastian,1,krytyk niezależny


In [6]:
result = []
for critic in tqdm.tqdm(df_critics['userName']):
    response = filmweb.get_reviewed_servies(critic).json()
    result += list(map(lambda review: {'username':critic, 'series_id':review[0]}, response))
df_critics_reviews_series = pd.DataFrame(result)
df_critics_reviews_series.head()

100%|██████████| 67/67 [00:11<00:00,  5.79it/s]


,username,series_id
0,michaloleszczyk,94606
1,michaloleszczyk,867132
2,michaloleszczyk,868514
3,michaloleszczyk,876630
4,michaloleszczyk,862322


In [7]:
it = 0
results = []
while len(results) < len(df_critics_reviews_series.values):
    for case in tqdm.tqdm(df_critics_reviews_series.values[it:]):
        try:
            response = filmweb.get_tv_series_review(critic_username=case[0], series_id=case[1]).json()
            if 'comment' in response:
                series_rating = filmweb.get_movie_rating(case[1]).json()
                results.append({'username': case[0], 
                                'series_id': case[1],
                                'user_rate': response['rate'],
                                'user_comment': response['comment'],
                                'series_rating': series_rating['rate'],
                                'series_rating_count': series_rating['count']
                                })
        except Exception as e:
            print(f'Fetching data about tv series rating failed: {e}')
            break
        it+=1
    if it >= len(df_critics_reviews_series.values):
        break

  7%|▋         | 262/3734 [03:08<41:35,  1.39it/s]


Fetching data about tv series rating failed: HTTPSConnectionPool(host='www.filmweb.pl', port=443): Max retries exceeded with url: /api/v1/user/makina13/vote/serial/843886 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f603320fc40>, 'Connection to www.filmweb.pl timed out. (connect timeout=None)'))


  9%|▉         | 329/3472 [04:30<43:02,  1.22it/s]  


Fetching data about tv series rating failed: HTTPSConnectionPool(host='www.filmweb.pl', port=443): Max retries exceeded with url: /api/v1/user/ksiawcik_13/vote/serial/809555 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f60329545e0>, 'Connection to www.filmweb.pl timed out. (connect timeout=None)'))


  2%|▏         | 73/3143 [02:25<1:41:54,  1.99s/it]


...

 27%|██▋       | 154/574 [02:45<07:32,  1.08s/it]


Fetching data about tv series rating failed: HTTPSConnectionPool(host='www.filmweb.pl', port=443): Max retries exceeded with url: /api/v1/user/mwol/vote/serial/99424 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f6032955990>, 'Connection to www.filmweb.pl timed out. (connect timeout=None)'))


 77%|███████▋  | 322/420 [03:35<01:05,  1.50it/s]


Fetching data about tv series rating failed: HTTPSConnectionPool(host='www.filmweb.pl', port=443): Max retries exceeded with url: /api/v1/user/dem3000/vote/serial/654010 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f6032955c60>, 'Connection to www.filmweb.pl timed out. (connect timeout=None)'))


100%|██████████| 98/98 [00:28<00:00,  3.50it/s]


In [8]:
ds_df = pd.DataFrame(results)
ds_df.head()

,username,series_id,user_rate,user_comment,series_rating,series_rating_count
0,michaloleszczyk,867132,8,NH. Grand finale mock-satanistycznej komedii V...,7.57339,436
1,michaloleszczyk,862322,10,Koronkowe komiczne eseje dokumentalne o urokac...,8.27645,4178
2,michaloleszczyk,819951,7,"Znakomicie zrealizowany serial, którego ostatn...",7.85234,10409
3,tru3,10011884,8,Ale to jest dobre! Absolutnie zaskakujący seri...,7.62731,7199
4,tru3,860894,5,Serial pozerski. Główny bohater - Czuły (Borys...,6.07103,4885


In [10]:
ds_df.to_json('../dataset/dataset_tv_series.json')
ds_df.to_csv('../dataset/dataset_tv_series.csv', sep='$', encoding='utf-8')

# Datasets merge

In [51]:
df_movies = pd.read_csv('../dataset/dataset_movies.csv', encoding='utf-8', sep='$')
df_tv_series = pd.read_csv('../dataset/dataset_series2.csv', encoding='utf-8', sep='$')

In [52]:
df_movies = df_movies.rename(axis='columns', mapper={'movie_id': 'show_id', 
                                                     'movie_rating': 'show_rating', 
                                                     'movie_rating_count': 'show_rating_count'})
df_movies.head()

,username,show_id,user_rate,user_comment,show_rating,show_rating_count
0,michaloleszczyk,10014103,8,Willa Hössa jako makieta najciemniejszej stron...,8.32095,296
1,michaloleszczyk,10025735,3,"Zgentryfikowana ""Mama i dziwka"" o smaku pumpki...",6.41777,833
2,michaloleszczyk,805204,9,"""Ile wilków widzisz na obrazku?"" A ile widzisz...",7.41684,9884
3,michaloleszczyk,10034333,9,Film bliski perfekcji; czysty i syty jak ronde...,7.36086,557
4,michaloleszczyk,10015490,9,Wielka niespodzianka i najbardziej oryginalny ...,5.44533,375


In [53]:
df_tv_series = df_tv_series.rename(axis='columns', mapper={'series_id': 'show_id', 
                                                           'series_rating': 'show_rating', 
                                                           'series_rating_count': 'show_rating_count'})
df_tv_series.head()

,username,show_id,user_rate,user_comment,show_rating,show_rating_count
0,michaloleszczyk,867132,8,NH. Grand finale mock-satanistycznej komedii V...,7.57339,436
1,michaloleszczyk,862322,10,Koronkowe komiczne eseje dokumentalne o urokac...,8.27645,4178
2,michaloleszczyk,819951,7,"Znakomicie zrealizowany serial, którego ostatn...",7.85234,10409
3,tru3,10011884,8,Ale to jest dobre! Absolutnie zaskakujący seri...,7.62731,7199
4,tru3,860894,5,Serial pozerski. Główny bohater - Czuły (Borys...,6.07103,4885


In [54]:
df_movies['type'] = 'movie'
df_tv_series['type'] = 'tv_series'

In [55]:
df_dataset = pd.concat([df_movies, df_tv_series], axis=0)
df_dataset

,username,show_id,user_rate,user_comment,show_rating,show_rating_count,type
0,michaloleszczyk,10014103,8,Willa Hössa jako makieta najciemniejszej stron...,8.32095,296,movie
1,michaloleszczyk,10025735,3,"Zgentryfikowana ""Mama i dziwka"" o smaku pumpki...",6.41777,833,movie
2,michaloleszczyk,805204,9,"""Ile wilków widzisz na obrazku?"" A ile widzisz...",7.41684,9884,movie
3,michaloleszczyk,10034333,9,Film bliski perfekcji; czysty i syty jak ronde...,7.36086,557,movie
4,michaloleszczyk,10015490,9,Wielka niespodzianka i najbardziej oryginalny ...,5.44533,375,movie
...,...,...,...,...,...,...,...
1326,alekhudzik,849662,3,O wow. Nie udało się zekranizować gry komputer...,7.77990,58245,tv_series
1327,alekhudzik,10012801,6,"No takie ten. ""1 świat 3 świat"", kalejdoskop p...",6.79320,353,tv_series
1328,alekhudzik,875491,7,Dwójka lepsza od jedynki. Zamykamy dyskusję,7.67484,40515,tv_series
1329,alekhudzik,877025,10,"Jest to najlepszy serial ""Coming of age"" świat...",7.22280,579,tv_series


In [57]:
df_dataset = df_dataset.reset_index(drop=True)

In [59]:
df_dataset.to_csv('../dataset/dataset.csv', encoding='utf-8', sep='$', index=False)
df_dataset.to_json('../dataset/dataset.json')

# Dataset cleanup
Filter out empty comments

In [84]:
df_dataset = pd.read_csv('../dataset/dataset.csv', sep='$', encoding='utf-8')
df_dataset

,username,show_id,user_rate,user_comment,show_rating,show_rating_count,type
0,michaloleszczyk,10014103,8,Willa Hössa jako makieta najciemniejszej stron...,8.32095,296,movie
1,michaloleszczyk,10025735,3,"Zgentryfikowana ""Mama i dziwka"" o smaku pumpki...",6.41777,833,movie
2,michaloleszczyk,805204,9,"""Ile wilków widzisz na obrazku?"" A ile widzisz...",7.41684,9884,movie
3,michaloleszczyk,10034333,9,Film bliski perfekcji; czysty i syty jak ronde...,7.36086,557,movie
4,michaloleszczyk,10015490,9,Wielka niespodzianka i najbardziej oryginalny ...,5.44533,375,movie
...,...,...,...,...,...,...,...
4383,alekhudzik,849662,3,O wow. Nie udało się zekranizować gry komputer...,7.77990,58245,tv_series
4384,alekhudzik,10012801,6,"No takie ten. ""1 świat 3 świat"", kalejdoskop p...",6.79320,353,tv_series
4385,alekhudzik,875491,7,Dwójka lepsza od jedynki. Zamykamy dyskusję,7.67484,40515,tv_series
4386,alekhudzik,877025,10,"Jest to najlepszy serial ""Coming of age"" świat...",7.22280,579,tv_series


In [85]:
df_dataset = df_dataset[~df_dataset['user_comment'].isna()].reset_index(drop=True)
df_dataset = df_dataset[df_dataset['user_comment'].str.len() > 3].reset_index(drop=True)
df_dataset

,username,show_id,user_rate,user_comment,show_rating,show_rating_count,type
0,michaloleszczyk,10014103,8,Willa Hössa jako makieta najciemniejszej stron...,8.32095,296,movie
1,michaloleszczyk,10025735,3,"Zgentryfikowana ""Mama i dziwka"" o smaku pumpki...",6.41777,833,movie
2,michaloleszczyk,805204,9,"""Ile wilków widzisz na obrazku?"" A ile widzisz...",7.41684,9884,movie
3,michaloleszczyk,10034333,9,Film bliski perfekcji; czysty i syty jak ronde...,7.36086,557,movie
4,michaloleszczyk,10015490,9,Wielka niespodzianka i najbardziej oryginalny ...,5.44533,375,movie
...,...,...,...,...,...,...,...
4276,alekhudzik,849662,3,O wow. Nie udało się zekranizować gry komputer...,7.77990,58245,tv_series
4277,alekhudzik,10012801,6,"No takie ten. ""1 świat 3 świat"", kalejdoskop p...",6.79320,353,tv_series
4278,alekhudzik,875491,7,Dwójka lepsza od jedynki. Zamykamy dyskusję,7.67484,40515,tv_series
4279,alekhudzik,877025,10,"Jest to najlepszy serial ""Coming of age"" świat...",7.22280,579,tv_series


In [ ]:
df_dataset.to_csv('../dataset/dataset.csv', encoding='utf-8', sep='$', index=False)
df_dataset.to_json('../dataset/dataset.json')